In [1]:
import os
os.chdir("../..")
import pandas as pd
import numpy as np
import math
import pdfplumber
import pandas as pd
from collections import defaultdict
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
project_path = os.getcwd()
listdir = ["/data/tourism/fiji/" +
           i for i in os.listdir(project_path + "/data/tourism/fiji")]

orig_file_num = [".xlsx" in file for file in listdir].index(True)

In [3]:
raw_t1 = pd.read_excel(
    project_path + listdir[orig_file_num], sheet_name="T1", usecols="B:N")

colnames_t1 = list()
for name in raw_t1.iloc[4].to_list():
    if type(name) == str:
        name = name.strip().lower()
        colnames_t1.append(name)
    else:
        colnames_t1.append("month")
raw_t1.columns = colnames_t1

fiji_t1 = (raw_t1.dropna(axis="columns", how="all")
           .iloc[5:-2].dropna(axis=0, how="all")
           .reset_index()
           .drop("index", axis=1))

In [5]:
def transform_dates(df: pd.DataFrame,
                    year_column: str,
                    start_year: int,
                    month_column: str):
    year_lst, dates = list(), list()

    for idx, val in enumerate(df[year_column].to_list()):
        year = (idx // 12) + start_year
        year_lst.append(year)
    df["year"] = year_lst

    for idx in df.index:
        month_raw = df[month_column][idx]
        if type(month_raw) is str:
            month = datetime.datetime.strptime(month_raw.strip(), "%B").month
            if month < 10:
                month = str(0) + str(month)
            year = df["year"][idx]
            dates.append(str(year) + "-" + str(month) + "-01")
    df["dates"] = dates

In [6]:
# Yearly data to 2020 FROM T1
fiji_t1_yearly = (fiji_t1.iloc[0:5, [0, 3, 4]]
                  .rename({"year/month": "year"}, axis=1))

fiji_t1_yearly.to_csv(str(project_path) + "/data/tourism/fiji/fj_num_visitors_yearly_17_21.csv",
                      encoding="utf-8")


# Monthly data from 2020 FROM T1
visitor_t1 = (fiji_t1.iloc[5:, [0, 1, 3, 4]]
              .reset_index()
              .drop("index", axis=1))

transform_dates(visitor_t1, "year/month", 2020, "month")
visitor_t1.to_csv(str(project_path) + "/data/tourism/fiji/fj_num_visitors.csv", encoding="utf-8")

In [7]:
raw_t2 = (pd.read_excel(project_path + listdir[0], sheet_name="T2", usecols="A:Q")
          .dropna(axis=0, how="all")
          .reset_index()
          .drop("index", axis=1))

# rename the columns
colnames_t2 = list()
for name1, name2 in zip(raw_t2.iloc[2].to_list(), raw_t2.iloc[3].to_list()):
    if type(name1) == str:
        wholename = name1.lower().strip() + name2.lower().strip()
        colnames_t2.append(wholename)
    else:
        if type(name2) == str:
            colnames_t2.append(name2.lower().strip())
        else:
            colnames_t2.append("month")
raw_t2.columns = colnames_t2

# Yearly data to 2020 FROM T2
fiji_t2_yearly = (raw_t2.iloc[4:9]
                  .reset_index()
                  .drop("index", axis=1))

fiji_t2_yearly.to_csv(str(project_path) + "/data/tourism/fiji/fj_num_visitors_by_origin_17_21.csv",
                      encoding="utf-8")

# Monthly data from 2020 FROM T2
fiji_t2 = (raw_t2.dropna(axis="columns", how="all")
           .iloc[9:-2].dropna(axis=0, how="all")
           .reset_index()
           .drop("index", axis=1))
transform_dates(fiji_t2, "year/month", 2020, "month")
fiji_t2 = fiji_t2.drop(["year/month", "month"], axis=1)

cols = fiji_t2.columns.to_list()
cols = cols[-2:] + cols[:-2]
fiji_t2 = fiji_t2[cols]
fiji_t2.to_csv(str(project_path) + "/data/tourism/fiji/fj_num_visitors_by_origin_20_by_month.csv",
               encoding="utf-8")

IsADirectoryError: [Errno 21] Is a directory: '/Users/czhang/Desktop/pacific-observatory/data/tourism/fiji/scraping'